In [2]:
import sys

sys.path.append('..')

In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

# no line wrap
pd.set_option('display.expand_frame_repr', False)

data = 'human'
channels = '*'
limit = 10000
batch_size = 64
lrs = [1e-3, 5e-4, 1e-4]
weight_decays = [0, 1e-4]
epochs = 20

# Define the directory and file pattern
# name = f'{model_name}_{args.data}_{args.channels}_{args.limit}_{args.num_moves}_{args.engine_prob}_{args.batch_size}_{args.lr}_{args.weight_decay}_{args.epochs}.csv'
# example = ConvLSTM_synthetic_12_10000_60_0.5_64_0.001_0.0001_20.csv
# pattern = os.path.join('../results', '*_processed_6_10000_40_*_64_*_*.csv')
pattern = os.path.join('../results', f'*_{data}_{channels}*.csv')
files = glob.glob(pattern)

results = {}


for file in files:
  data = file.split('/')[-1].split('_')
  model = data[0]
  lr = float(data[-3])
  weight_decay = float(data[-2])
  channels = int(data[2])
  df = pd.read_csv(file)
  df = df[df['type'] == 'test']
  row = df[df['accuracy'] == df['accuracy'].max()].iloc[0].to_dict()
  del row['epoch']
  del row['loss']
  del row['type']
  row['lr'] = lr
  row['wd'] = weight_decay
  row['channels'] = int(channels)
  if model not in results:
    results[model] = row
  else:
    if row['accuracy'] > results[model]['accuracy']:
      results[model] = row

df = pd.DataFrame(results).T
df = df.sort_values('accuracy', ascending=False)
print(df)

                           accuracy precision    recall        f1       auc    dataset      lr      wd channels
Transformer                0.518182  0.452586  0.518182   0.47875  0.503817  generated   0.001  0.0001       12
Transformer(evals)              0.4  0.414406       0.4  0.403136  0.433049  generated  0.0005  0.0001       12
ConvLSTM(times)            0.336364  0.454321  0.336364  0.372559  0.463745  generated  0.0005     0.0        6
ConvLSTM(evals)            0.327273   0.40003  0.327273  0.354387  0.391362  generated  0.0005  0.0001        6
ConvLSTM(evals, times)     0.309091  0.394034  0.309091  0.341627  0.395307  generated  0.0005  0.0001       12
ConvLSTM                        0.3  0.388996       0.3  0.329359  0.464727  generated   0.001     0.0       12
Transformer(evals, times)  0.290909  0.367197  0.290909  0.301175  0.434562  generated   0.001     0.0        6
Transformer(times)         0.254545  0.406482  0.254545  0.260963  0.471496  generated   0.001     0.0  